# Scenario


**You can change or extend your own code as you wish to cover new features or API. However, if you want to use someone's libraies, you usally have to take them as they are.**


**Solutions** to alleviate this problem have sprung up in programming languages
 1. Ruby has modules
 2. Smalltalk let pacakges add to each other's class
 3. C\#3.0 has static extension modules, which are more local but also more restrictive, in that you can only add methods, not fields, to a class, and you cannot make a class implement new interfaces.
 4. For Bytecode programming langauge, we can also use code imstrumentation to insert a function to some specicial position in the code to expose some information or get current state of runing code. 
 4. Scala's answer is implicit conversions and parameters.

## What is implicit conversion

1. A value can be passed “automatically” to the definition of a method/class via implicit parameters
2. Implicits definitions are those that the compiler is allowed to insert into a **program** in order to fix any of its type errors. For example, if *** x + y *** is not compiled well and has a type mismatch error on *x* since there is no operator *+* defined in *x*. So the compiler meight change it to ***convert(x) + y***, where *convert* is some available implicit convertion to change *x* to something that has a *+* method, then this change might fix a program so that it can type checks and return correctly. You should be noticed that the coversion should work in a specific scope,  which means you need to define them in a context of code.

# Rules for imlicits

## Making Rules

**Only defitions marked by *implict* are available.** 
1. You can use it to mark any variable, function, class or object definition
2. Variables and singleton objects marked implicit can be used as *implicit parameters*. 


In [3]:
// you will get a runtime error, since there is a type mismatch between a integer and string types.
2 + "abbc"

res2: String = "2abbc"

In [4]:
implicit def intToString(x:Int) = x.toString

defined function intToString

In [5]:
2 + "abbc"

res4: String = "2abbc"

In the inner compiler, the above expression is parsed and represented by *intToString(2) + "abbc")* acturally.

## Scope Rule

**An inserted implicit conversion must be in scope as a single identifier, or be associated with the source or target type of the coversion**

There are two kinds of places that compiler will look for the avaialble implicit definitions:

1. The current working scope. Therefore, you must in some way bring the definitions into the scope. For example you can import them from other packages.  
2. The implicit conversion must be in scope as a **single** identifier. For example, *someVarible.convert* is not inserted by compiler to expand x to *someVarible.convert(x) + y*.

defined function intToString

3. There's one exception to the "single identifier" rule. The compiler will aso look for implicit definitions in the companion object of the source or expected target types of the coversion. 

In [2]:
abstract class currency(n:Float){
    val nums = n
}

class Euro(n:Float) extends currency(n)

class Dollar(n:Float) extends currency(n){
    def add(a: Dollar) = new Dollar(this.nums + a.nums)
}

defined class currency
defined class Euro
defined class Dollar

In [3]:
val a_euro = new Euro(10)

a_euro: Euro = ammonite.$sess.cmd1$Helper$Euro@282ee7a4

In [4]:
val a_dollar = new Dollar(15)

a_dollar: Dollar = ammonite.$sess.cmd1$Helper$Dollar@3fb3ea15

In [4]:
a_dollar add(a_euro)

cmd4.sc:1: type mismatch;
 found   : ammonite.$sess.cmd2.wrapper.cmd1.Euro
 required: ammonite.$sess.cmd3.wrapper.cmd1.Dollar
val res4 = a_dollar add(a_euro)
                        ^Compilation Failed

: 

There is a type mismatch issue here, since the add function in Dollar can only take Dollar class. So there are two ways to fix this issue.

1. In the companion object of target class, that is "Dollar", you can define an implicit conversion 

In [7]:
abstract class currency(n:Float){
    val nums = n
}

class Euro(n:Float) extends currency(n)

class Dollar(n:Float) extends currency(n){
    def add(a: Dollar) = new Dollar(this.nums + a.nums)
}

object Dollar{
    implicit def dollarToEuro(x: Euro) = new Dollar(x.nums)
}

val a_euro = new Euro(10)
val a_dollar = new Dollar(15)

defined class currency
defined class Euro
defined class Dollar
defined object Dollar
a_euro: Euro = ammonite.$sess.cmd6$Helper$Euro@32baf94d
a_dollar: Dollar = ammonite.$sess.cmd6$Helper$Dollar@5b935dfa

In [8]:
a_dollar add(a_euro)

res7: Dollar = ammonite.$sess.cmd6$Helper$Dollar@6bafad15

2. In the companion object of source class, that is "Euro", you can define an implicit conversion 

In [9]:
abstract class currency(n:Float){
    val nums = n
}

class Euro(n:Float) extends currency(n)

class Dollar(n:Float) extends currency(n){
    def add(a: Dollar) = new Dollar(this.nums + a.nums)
}

object Euro{
    implicit def dollarToEuro(x: Euro) = new Dollar(x.nums)
}

val a_euro = new Euro(10)
val a_dollar = new Dollar(15)

defined class currency
defined class Euro
defined class Dollar
defined object Euro
a_euro: Euro = ammonite.$sess.cmd8$Helper$Euro@4912d8a1
a_dollar: Dollar = ammonite.$sess.cmd8$Helper$Dollar@2947c4bc

In [10]:
a_dollar add(a_euro)

res9: Dollar = ammonite.$sess.cmd8$Helper$Dollar@7555afe5

In above two cases, the coversion *dollarToEuro* is said to the *associated* to the type *Dollar* or *Euro*. The compiler will find such an associated conversion every time it needs to convert from an instance of type *Dollar*. There is no need to import it seperately into your program.

**You cannot define an implicit conversion in the class, See followed examples**

In [12]:
abstract class currency(n:Float){
    val nums = n
}

class Euro(n:Float) extends currency(n){
    implicit def dollarToEuro(x: Euro) = new Dollar(x.nums)
}

class Dollar(n:Float) extends currency(n){
    def add(a: Dollar) = new Dollar(this.nums + a.nums)
}
val a_euro = new Euro(10)
val a_dollar = new Dollar(15)

defined class currency
defined class Euro
defined class Dollar
a_euro: Euro = ammonite.$sess.cmd11$Helper$Euro@74446b7a
a_dollar: Dollar = ammonite.$sess.cmd11$Helper$Dollar@789f82ca

In [12]:
a_dollar add(a_euro)

cmd12.sc:1: type mismatch;
 found   : cmd12.this.cmd11.Euro
 required: cmd12.this.cmd11.Dollar
val res12 = a_dollar add(a_euro)
                         ^Compilation Failed

: 

There are some benefits from scope rule
1. it helps with modular reasoning. When you read code in a file, except for the context definiton in the code, the only things you need to consider from other files are those that are either imported or explicitly referrenced through a fully qualified name. 

## One-at-a-time rule

For *x + y*, the compiler will never rewrite it to *convert1(convert2(x)) + y*. For sanity's sake, the compiler does not insert further implicit conversions when it is already in the middle of trying another implicit. 

However, it is possible to **circumvent this restriction** by having implicits take implicit parameters. For example:  

In [12]:
//TODO

## Explicits-First Rule

**Whenever code type checks as it is written, no implicits are attempted** A corollary of this rule is that you can always replace implicit identifiers by explicit ones.

## Naming an implicit conversion

Implicit conversions can have arbitrary names. The name of an implicit conversion matters only in two situations:
1. if you want to write it explicitly in a method application
2. for determining which implicit conversions are available at any place in the program


To illustrate the second point, say you have an object with two implicit conversions:

In [12]:
object MyConversions {
    implicit def stringWrapper(s: String):IndexedSeq[Char] = ...
    implicit def intToString(x: Int): String = ...
}

(console):2:62 expected (If | While | Try | DoWhile | For | Throw | Return | ImplicitLambda | SmallerExprOrLambda)
object MyConversions {
    implicit def stringWrapper(s: String):IndexedSeq[Char] = ...
    implicit def intToString(x: Int): String = ...
}
                                                                                    ^

: 

In your application, you want to make use of the *stringWrapper* conversion, but you don’t want integers to be converted automatically to strings by means of the *intToString* conversion. You can achieve this by importing only one conversion, but not the other.

In [12]:
import MyConversions.stringWrapper
... // code making use of stringWrapper

(console):2:2 expected (`this` | Id)
import MyConversions.stringWrapper
... // code making use of stringWrapper
                                    ^

: 

## Where implicits are tried 
There are three places implicits are used in the language
1. **Implicit conversions to an expected type.** let you use one type in a context where a different type is expected. For example, you might have a String and want to pass it to a method that requires an *IndexedSeq[Char]*.

2. **Implicit Conversions of the receiver of a selection.** let you adapt the receiver of a method call, i.e., the object on which a method is invoked, if the method is not applicable on the original type. An example is *"abc".exists*, which is converted to *stringWrapper("abc").exists* because the exists method is not available on Strings but is available on *IndexedSeqs*

3. **Implicit parameters.** are usually used to provide more information to the called function about what the caller wants. Implicit parameters are especially useful with generic functions, where the called function might otherwise know nothing at all about the type of one or more arguments.

# Implicit conversion to an expected type


Whenever the compiler sees an **X**, but needs a **Y**, it will look for an implicit function that converts X to Y. 

Speaking very briefly about the latter type, if one calls a method m on an object o of a class C, and that class does not support method m, then Scala will look for an implicit conversion from C to something that does support m. 

In [17]:
// normally a double cannot be used as an integer, because it loses precision
val i:Int = 3.5

cmd17.sc:1: type mismatch;
 found   : Double(3.5)
 required: Int
val i:Int = 3.5
            ^Compilation Failed

: 

However, you can define an implicit conversion to smooth this over:

In [18]:
implicit def doubleToInt(x: Double) = x.toInt
val i: Int = 3.5

defined function doubleToInt
i: Int = 3

What happens here is that the compiler sees a Double, specifically 3.5, in a context where it requires an Int. So far, the compiler is looking at an ordinary type error. Before giving up, though, it searches for an implicit conversion from Double to Int. In this case, it finds one: doubleToInt, because doubleToInt is in scope as a single identifier. (Outside the interpreter, you might bring doubleToInt into scope via an import or possibly through inheritance.) The compiler then inserts a call to doubleToInt automatically. Behind the scenes, the code becomes:

In [19]:
val i: Int = doubleToInt(3.5) // This is literally an implicit conversion

i: Int = 3

Converting Doubles to Ints might raise some eyebrows, because it’s a dubious idea to have something that causes a loss in precision happen invisibly. So this is not really a conversion we recommend. **It makes much more sense to go the other way, from some more constrained type to a more general one**. For instance, an Int can be converted without loss of precision
to a Double, so an implicit conversion from Int to Double makes sense. In fact, that’s exactly what happens. The cala.Predef object, which is implicitly imported into every Scala program, defines implicit conversions that convert “smaller” numeric types to “larger” ones.

# Converting the receiver


Implicit conversions also apply to the receiver of a method call, the object on which the method is invoked. This kind of implicit conversion has two main uses.
1. Receiver conversions allow smoother integration of a new class into an existing class hierarchy
2. They support writing domainspecific languages (DSLs) within the language

To see how it works, suppose you write down obj.doIt, and obj does not have a member named doIt. The compiler will try to insert conversions before giving up. In this case, the conversion needs to apply to the receiver, obj. The compiler will act as if the expected “type” of obj were “has a member named doIt.” This “has a doIt” type is not a normal Scala type, but it is there conceptually and is why the compiler will insert an implicit conversion in this case.

## Interoperating with new types

As mentioned previously, one major use of receiver conversions is allowing smoother integration of new with existing types. In particular, they allow you to enable client programmers to use instances of existing types as if they were instances of your new type

In [1]:
class Rational(n: Int, d: Int) {
    require(d != 0)
    private val g = gcd(n.abs, d.abs)
    
    val numer = n / g
    val denom = d / g
    def this(n: Int) = this(n, 1)
    def + (that: Rational): Rational =
        new Rational(
            numer * that.denom + that.numer * denom,
            denom * that.denom
        )
    def + (i: Int): Rational = new Rational(numer + i * denom, denom)
    
    override def toString = numer +"/"+ denom
    private def gcd(a: Int, b: Int): Int = if (b == 0) a else gcd(b, a % b)
}

defined class Rational

In [2]:
val oneHalf = new Rational(1, 2)

oneHalf: Rational = 1/2

In [3]:
oneHalf + 1

res2: Rational = 3/2

In [3]:
// This expression is tricky because the receiver, 1, does not have a suitable + method working on a rational number. 
// So there is a compiling error
1 + oneHalf

cmd3.sc:1: overloaded method + with alternatives:
  (x: Double)Double <and>
  (x: Float)Float <and>
  (x: Long)Long <and>
  (x: Int)Int <and>
  (x: Char)Int <and>
  (x: Short)Int <and>
  (x: Byte)Int <and>
  (x: String)String
 cannot be applied to (ammonite.$sess.cmd1.wrapper.cmd0.Rational)
val res3 = 1 + oneHalf
             ^Compilation Failed

: 

To allow this kind of mixed arithmetic, you need to define an implicit conversion from Int to Rational

In [4]:
implicit def intToRational(x: Int) = new Rational(x, 1)

defined function intToRational

In [5]:
1 + oneHalf

res4: Rational = 3/2

What happens behind the scenes here is that Scala compiler first tries to type check the expression 1 + oneHalf as it is. This fails because Int has several + methods, but none that takes a Rational argument. Next, the compiler searches for an implicit conversion from Int to another type that has a + method which can be applied to a Rational. It finds your conversion and applies it, which yields:

In [6]:
intToRational(1) + oneHalf

res5: Rational = 3/2

## Simulating new syntax

The other major use of implicit conversions is to simulate adding new syntax. Recall that you can make a Map using syntax like this: 

In [7]:
val a = Map(1 -> "one", 2 -> "two", 3 ->"three")

a: Map[Int, String] = Map(1 -> "one", 2 -> "two", 3 -> "three")

In this definition, Have you wondered how the "->" is supported? 

Actually, it is not syntax. Instead "->" is a method of the class **ArrowAssoc**, a class defined inside the standard Scala preamble (scala.Predef). The preamble also defines an implicit conversion from Any to ArrowAssoc. When you write following statement:

In [10]:
1 -> "one"

res9: (Int, String) = (1, "one")

The compiler inserts a conversion from 1 to **ArrowAssoc** so that the "->" method can be found. 

```
package scala
object Predef {
class ArrowAssoc[A](x: A) {
    def ->[B](y: B): Tuple2[A, B] = Tuple2(x, y)
}
implicit def any2ArrowAssoc[A](x: A): ArrowAssoc[A] = new ArrowAssoc(x)
...
}
```

In [11]:
ArrowAssoc(1) -> "one"

res10: (Int, String) = (1, "one")

This “rich wrappers” pattern is common in libraries that provide syntax-like extensions to the language, so you should be ready to recognize the pattern when you see it. Whenever you see someone calling methods that appear not to exist in the receiver class, they are probably using implicits. Similarly, if you see a class named RichSomething, e.g., RichInt or RichBoolean, that class is likely adding syntax-like methods to type Something.

As you can now see, these rich wrappers apply more widely, often letting you get by with an internal DSL defined as a library where programmers in other languages might feel the need to develop an external DSL.

# Implicit Parameters


A method can have an implicit parameter list, marked by the implicit keyword at the start of the parameter list. If the parameters in that parameter list are not passed as usual, Scala will look if it can get an implicit value of the correct type, and if it can, pass it automatically.

The places Scala will look for these parameters fall into two categories:

1. Scala will first look for implicit definitions and implicit parameters that can be accessed directly (without a prefix) at the point the method with the implicit parameter block is called
2. Then it looks for members marked implicit in all the companion objects associated with the implicit candidate type.

In [16]:
import scala.math._
def foo[T](t: T)(implicit integral: Integral[T]): Unit = {
    println(integral)
}

import scala.math._

defined function foo

In [21]:
foo(0)

scala.math.Numeric$IntIsIntegral$@5d63bd3e


## Context Bounds

Another common pattern in implicit parameters is the type class pattern. This pattern enables the provision of common interfaces to classes which did not declare them. It can both serve as a bridge pattern – gaining separation of concerns – and as an adapter pattern.

In [23]:
def sum[T](list: List[T])(implicit integral: Integral[T]): T = {
    import integral._   // get the implicits in question into scope
    list.foldLeft(integral.zero)(_ + _)
}

defined function sum

There is also a syntactic sugar for it, called a context bound, which is made less useful by the need to refer to the implicit. A straight conversion of that method looks like this:

In [25]:
def sum[T : Integral](list: List[T]): T = {
    val integral = implicitly[Integral[T]]
    import integral._   // get the implicits in question into scope
    list.foldLeft(integral.zero)(_ + _)
}

defined function sum

Context bounds are more useful when you just need to pass them to other methods that use them. For example, the method *sorted* on *Seq* needs an implicit Ordering. To create a method reverseSort, one could write:

In [27]:
def reverseSort[T : Ordering](seq: Seq[T]) = seq.sorted.reverse

defined function reverseSort

Because Ordering[T] was implicitly passed to reverseSort, it can then pass it implicitly to sorted.

# Implicit Class

The implicit keyword will now be allowed as an annotation on classes. Classes annotated with the *implicit* keyword are referred to as *implicit classes*.

1. An implicit class must have a primary constructor **with exactly one argument** in its first parameter list. It may also include an additional implicit parameter list. 
2. An implicit class must be defined in a scope where method definitions are allowed (not at the **top** level).
3. An implicit class is desugared into **a class** and **implicit method** pairing, where the implicit method mimics the constructor of the class.

For example: 

In [11]:
implicit class RichInt(n: Int) extends Ordered[Int] {
    def min(m: Int): Int = if (n <= m) n else m
    ...
}

(console):3:6 expected (`this` | Id)
implicit class RichInt(n: Int) extends Ordered[Int] {
    def min(m: Int): Int = if (n <= m) n else m
    ...
}
                                                                                                           ^

: 

will be transformed by the compiler as follows:

In [11]:
class RichInt(n: Int) extends Ordered[Int] {
  def min(m: Int): Int = if (n <= m) n else m
  ...
}
implicit final def RichInt(n: Int): RichInt = new RichInt(n)

(console):3:4 expected (`this` | Id)
class RichInt(n: Int) extends Ordered[Int] {
  def min(m: Int): Int = if (n <= m) n else m
  ...
}
implicit final def RichInt(n: Int): RichInt = new RichInt(n)
                                                                                              ^

: 

The generated implicit method(**RichInt**) will have the same name as the implicit class(**RichInt**). This allows importing the implicit conversion using the name of the class, as one expects from other implicit definitions

4. Annotations on implicit classes default to attaching to the generated class and the method. For example:

In [11]:
@bar
implicit class Foo(n: Int)

cmd11.sc:1: not found: type bar
@bar
 ^Compilation Failed

: 

Will desugar into:

In [11]:
@bar class Foo(n:Int)
@bar implicit def Foo(n: Int): Foo = new Foo(n)

cmd11.sc:1: not found: type bar
@bar class Foo(n:Int)
 ^cmd11.sc:2: not found: type bar
@bar implicit def Foo(n: Int): Foo = new Foo(n)
 ^Compilation Failed

: 

5. The annotation.target annotations will be expanded to include a genClass and method annotation. This can be used to target annotations at just the generated class or the generated method of an implicit class. For example:

In [11]:
@(bar @genClass) implicit class Foo(n: Int)

cmd11.sc:1: not found: type bar
@(bar @genClass) implicit class Foo(n: Int)
  ^cmd11.sc:1: not found: type genClass
@(bar @genClass) implicit class Foo(n: Int)
       ^Compilation Failed

: 

will desugar into

In [11]:
implicit def Foo(n: Int): Foo = new Foo(n)
@bar class Foo(n: Int)

cmd11.sc:2: not found: type bar
@bar class Foo(n: Int)
 ^Compilation Failed

: 

## Restrictions

Implicit classes have the following restrictions:

1. They must be defined inside of another *trait/class/object*

In [14]:
object Helpers {
   implicit class RichInt(x: Int) // OK!
}
implicit class RichDouble(x: Double) // BAD!

defined object Helpers
defined class RichDouble

2. They may only take one non-implicit argument in their constructor, but can take multiple implicit parameters

In [14]:
implicit class RichDate(date: java.util.Date) // OK!
implicit class IndexerBad[T](collection: Seq[T], index: Int) // BAD!
implicit class IndexerOK[T](collection: Seq[T])(implicit index: Int) // OK, with one implicit parameter

cmd14.sc:2: implicit classes must accept exactly one primary constructor parameter
implicit class IndexerBad[T](collection: Seq[T], index: Int) // BAD!
               ^Compilation Failed

: 

While it’s possible to create an implicit class with more than one non-implicit argument, such classes aren’t used during implicit lookup.

3. There may not be any method, member or object in scope with the same name as the implicit class. This means an implicit class cannot be a case class.

```
object Bar
implicit class Bar(x: Int) // BAD!

val x = 5
implicit class x(y: Int) // BAD!

implicit case class Baz(x: Int) // BAD!
```

# Implicit conversions as implicit parameters
There’s one situation where an implicit is both an implicit conversion and an implicit parameter. For example:

In [20]:
def getIndex[T, CC](seq: CC, value: T)(implicit conv: CC => Seq[T]) = seq.indexOf(value)

getIndex("abc", 'a')

defined function getIndex
res19_1: Int = 0

The method *getIndex* can receive any object, as long as there is an implicit conversion available from its class to *Seq[T]*. Because of that, a String can be passed to *getIndex*, and it will work.

Behind the scenes, the compiler changes *seq.IndexOf(value)* to *conv(seq).indexOf(value)*

# Where do Implicits Come From?

## Implicits Defined in Lexical Scope
When a value of a certain name is required, lexical scope is searched for a value with that name. Similarly, when an implicit value of a certain type is required, lexical scope is searched for a value with that type.

For example, here is a function that takes an implicit scaling factor. The function requires a parameter of type Int, and there is a value of that type in scope. The variable name n does not matter in this case.


In [28]:
implicit val n: Int = 5
def scale(x: Int)(implicit y: Int) = x * y
scale(5) // takes n from the current scope, with the result 25

n: Int = 5
defined function scale
res27_2: Int = 25

The invocation can be rewritten scale(5)(n). If n can be referenced using its simple name, as shown here, it is eligible as an implicit value.

An implicit value can be introduced into scope by an import statement:

In [29]:
import scala.collection.JavaConverters._
def env = System.getenv().asScala   // extension method enabled by imported implicit
val term = env("TERM")              // it's a Scala Map

: 

## Implicits Defined in Implicit Scope
Implicit syntax can avoid the import tax, which of course is a “sin tax,” by leveraging “implicit scope”, which depends on the type of the implicit instead of imports in lexical scope.

When an implicit of type T is required, implicit scope includes the companion object T:

In [30]:
trait T
object T { implicit val t: T = new T { } }

defined trait T
defined object T

# Reference

1. [WHERE DOES SCALA LOOK FOR IMPLICITS?](https://docs.scala-lang.org/tutorials/FAQ/finding-implicits.html)
2. [IMPLICIT PARAMETERS](https://docs.scala-lang.org/tour/implicit-parameters.html)